# Robot Arm Torque Analysis

This notebook performs torque analysis for a robot arm using forward kinematics, inverse kinematics, Jacobians, and dynamics equations. We will calculate the joint torques required for a given task.

## Imports and Definitions
We start by importing necessary libraries and defining the forward kinematics, inverse kinematics, and Jacobians.


In [1]:
import sympy as sp
import math
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

In [4]:
# Define symbolic variables for angles and dimensions
theta_1, theta_2, theta_3, theta_4, theta_5 = sp.symbols("theta_1 theta_2 theta_3 theta_4 theta_5")
d_1, d_5 = sp.symbols("d_1 d_5")
a_2, a_3 = sp.symbols("a_2 a_3")
alpha = [90, 0, 0, 90, 0]

# Helper function to create a transformation matrix from DH parameters
def dh_matrix(theta, d, a, alpha):
    alpha_rad = sp.rad(alpha)  # Convert alpha from degrees to radians
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha_rad), sp.sin(theta) * sp.sin(alpha_rad), a * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha_rad), -sp.cos(theta) * sp.sin(alpha_rad), a * sp.sin(theta)],
        [0, sp.sin(alpha_rad), sp.cos(alpha_rad), d],
        [0, 0, 0, 1]
    ])

In [5]:

# Create transformation matrices for each joint using the updated parameters
A1 = dh_matrix(theta_1, d_1, 0, alpha[0])
A2 = dh_matrix(theta_2, 0, a_2, alpha[1])
A3 = dh_matrix(theta_3, 0, a_3, alpha[2])
A4 = dh_matrix(theta_4, 0, 0, alpha[3])  # a_4 is zero
A5 = dh_matrix(theta_5, d_5, 0, alpha[4])  # a_5 is zero, added d_5

# Compute the overall transformation matrix by multiplying individual matrices
T = A1 * A2 * A3 * A4 * A5

# Extract the position and orientation
position = T[:3, 3]
orientation = T[:3, :3]

# Compute the Jacobian for linear velocity
Jv = position.jacobian([theta_1, theta_2, theta_3, theta_4, theta_5])

# The angular part of the Jacobian is given by the z-axis of the previous frames
z0 = sp.Matrix([0, 0, 1])  # z0 axis (base frame)
z1 = A1[:3, :3] * z0
z2 = (A1 * A2)[:3, :3] * z0
z3 = (A1 * A2 * A3)[:3, :3] * z0
z4 = (A1 * A2 * A3 * A4)[:3, :3] * z0

# Assemble the angular velocity Jacobian
Jw = sp.Matrix.hstack(z0, z1, z2, z3, z4)

# Combine Jv and Jw to form the full Jacobian matrix
J_full = sp.Matrix.vstack(Jv, Jw)

## Dynamic Parameters and Torque Calculation
Next, we define the dynamic parameters and compute the joint torques required.


In [6]:
# Define symbolic variables for dynamic parameters
M11, M12, M13, M14, M15 = sp.symbols("M11 M12 M13 M14 M15")
M21, M22, M23, M24, M25 = sp.symbols("M21 M22 M23 M24 M25")
M31, M32, M33, M34, M35 = sp.symbols("M31 M32 M33 M34 M35")
M41, M42, M43, M44, M45 = sp.symbols("M41 M42 M43 M44 M45")
M51, M52, M53, M54, M55 = sp.symbols("M51 M52 M53 M54 M55")

C11, C12, C13, C14, C15 = sp.symbols("C11 C12 C13 C14 C15")
C21, C22, C23, C24, C25 = sp.symbols("C21 C22 C23 C24 C25")
C31, C32, C33, C34, C35 = sp.symbols("C31 C32 C33 C34 C35")
C41, C42, C43, C44, C45 = sp.symbols("C41 C42 C43 C44 C45")
C51, C52, C53, C54, C55 = sp.symbols("C51 C52 C53 C54 C55")

G1, G2, G3, G4, G5 = sp.symbols("G1 G2 G3 G4 G5")

# Define joint accelerations and external forces
ddq1, ddq2, ddq3, ddq4, ddq5 = sp.symbols("ddq1 ddq2 ddq3 ddq4 ddq5")
dq1, dq2, dq3, dq4, dq5 = sp.symbols("dq1 dq2 dq3 dq4 dq5")
Fx, Fy, Fz, Mx, My, Mz = sp.symbols("Fx Fy Fz Mx My Mz")

# Define dynamic parameter matrices
M = sp.Matrix([
    [M11, M12, M13, M14, M15],
    [M21, M22, M23, M24, M25],
    [M31, M32, M33, M34, M35],
    [M41, M42, M43, M44, M45],
    [M51, M52, M53, M54, M55]
])

C = sp.Matrix([
    [C11, C12, C13, C14, C15],
    [C21, C22, C23, C24, C25],
    [C31, C32, C33, C34, C35],
    [C41, C42, C43, C44, C45],
    [C51, C52, C53, C54, C55]
])

G = sp.Matrix([G1, G2, G3, G4, G5])

# Combine joint velocities and accelerations
ddq = sp.Matrix([ddq1, ddq2, ddq3, ddq4, ddq5])
dq = sp.Matrix([dq1, dq2, dq3, dq4, dq5])

# External forces and torques
F_ext = sp.Matrix([Fx, Fy, Fz, Mx, My, Mz])

# Compute the required torques
tau = M * ddq + C * dq + G - J_full.T * F_ext

# Simplify the result
tau_simplified = tau.simplify()

# Print the simplified torques
sp.init_printing(use_unicode=True)
print("Simplified Joint Torques:")
sp.pprint(tau_simplified)


## Numerical Evaluation
We will now evaluate the torques numerically with given values for dynamic parameters and joint states.


In [ ]:
# Example numerical values for the dynamic parameters
M_val = np.array([
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
])

C_val = np.zeros((5, 5))
G_val = np.array([0, 0, 9.81, 0, 0])

# Example joint states
ddq_val = np.array([0, 0, 0, 0, 0])  # Zero accelerations for static analysis
dq_val = np.array([0, 0, 0, 0, 0])   # Zero velocities for static analysis
F_ext_val = np.array([0, 0, 0, 0, 0, 0])  # No external forces

# Compute the torques numerically
J_full_val = np.array(J_full).astype(np.float64)  # Convert symbolic to numerical
tau_val = M_val @ ddq_val + C_val @ dq_val + G_val - J_full_val.T @ F_ext_val

print("Required Joint Torques:", tau_val)


## Robot Arm Class with Inverse Kinematics and Plotting
We define a class for the robot arm that includes inverse kinematics and plotting functionality.


In [ ]:
class RobotArm:
    def __init__(self, d1, a2, a3, d5):
        self.d1 = d1
        self.a2 = a2
        self.a3 = a3
        self.d5 = d5

    def inverse_kinematics(self, Px, Py, Pz, omega):
        R = self.d5 * math.cos(math.radians(omega))
        theta1 = math.degrees(math.atan2(Py, Px))

        Pxw = Px - R * math.cos(math.radians(theta1))
        Pyw = Py - R * math.sin(math.radians(theta1))
        Pzw = Pz + self.d5 * math.sin(math.radians(omega))

        Rw = math.sqrt(Pxw**2 + Pyw**2)
        S = math.sqrt((Pzw - self.d1) ** 2 + Rw**2)

        alpha = math.degrees(math.atan2(Pzw - self.d1, Rw))
        beta = math.degrees(math.acos((self.a2**2 + S**2 - self.a3**2) / (2 * self.a2 * S)))

        theta2 = alpha + beta
        theta2_alt = alpha - beta

        theta3 = math.degrees(math.acos((S**2 - self.a2**2 - self.a3**2) / (2 * self.a2 * self.a3)))
        theta3 = -theta3

        theta234 = 90 - omega
        theta4 = theta234 - theta2 - theta3

        return theta1, theta2, theta3, theta4

    def forward_kinematics(self, theta1, theta2, theta3, theta4):
        theta1 = math.radians(theta1)
        theta2 = math.radians(theta2)
        theta3 = math.radians(theta3)
        theta4 = math.radians(theta4)

        x0, y0, z0 = 0, 0, 0
        x1, y1, z1 = 0, 0, self.d1
        x2 = self.a2 * math.cos(theta1) * math.cos(theta2)
        y2 = self.a2 * math.sin(theta1) * math.cos(theta2)
        z2 = self.d1 + self.a2 * math.sin(theta2)
        x3 = x2 + self.a3 * math.cos(theta1) * math.cos(theta2 + theta3)
        y3 = y2 + self.a3 * math.sin(theta1) * math.cos(theta2 + theta3)
        z3 = z2 + self.a3 * math.sin(theta2 + theta3)

        x4 = x3 + self.d5
        y4 = y3
        z4 = z3

        return [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2), (x3, y3, z3), (x4, y4, z4)]

    def plot_robot(self, joint_positions):
        x, y, z = zip(*joint_positions)

        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
        ax.plot(x, y, z, "o-", markersize=10, label="Robot Arm")
        ax.scatter(x, y, z, c="k")

        for i in range(len(joint_positions) - 1):
            ax.text(
                (x[i] + x[i + 1]) / 2,
                (y[i] + y[i + 1]) / 2,
                (z[i] + z[i + 1]) / 2,
                f"Link {i + 1}",
                color="black"
            )

        ax.set_xlabel("X axis")
        ax.set_ylabel("Y axis")
        ax.set_zlabel("Z axis")
        ax.set_title("3D Robot Configuration")
        ax.legend()
        ax.set_box_aspect([1, 1, 1])
        plt.show()

# Example parameters
Px, Py, Pz = 1, 0, 0
omega = 0

# Define DH parameters
d1, a2, a3, d5 = 0.1, 0.5, 0.5, 0.1

robot_arm = RobotArm(d1, a2, a3, d5)
theta1, theta2, theta3, theta4 = robot_arm.inverse_kinematics(Px, Py, Pz, omega)
joint_positions = robot_arm.forward_kinematics(theta1, theta2, theta3, theta4)
robot_arm.plot_robot(joint_positions)
